# Detectando objetos com YOLO v4 e OpenCV

## Visão geral

No notebook anterior, utilizamos o framework Darknet para realizar a detecção de objetos em imagens com o You only look once (YOLO). Entretanto, também é possível empregar o módulo cv2, que corresponde à implementação do OpenCV em Python, para executar a mesma tarefa, sem a necessidade de compilar (buildar) o framework Darknet.

## Etapa 1 - Importando as bibliotecas

In [3]:
import cv2
import numpy as np
import time

print(cv2.__version__)

4.12.0


## Etapa 2 - Conectando com o Google Drive

In [4]:
# Desnecessario

## Etapa 3 - Carregando os arquivos do modelo treinado

In [ ]:
import utils

# Obter pesos, configuracao da arquitetura e labels da rede neural do modelo
yolov3 = utils.carregar_yolov3()
yolov3

yolov3(labels_path='/home/rapha/Projetos/opencv-yolo-darknet/shared/modelo_YOLOv3/coco.names', weights_path='/home/rapha/Projetos/opencv-yolo-darknet/shared/modelo_YOLOv3/yolov3.weights', config_path='/home/rapha/Projetos/opencv-yolo-darknet/shared/modelo_YOLOv3/yolov3.cfg', labels=['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 

In [ ]:
# Carregar Rede Neural Profunda (DNN)
net = cv2.dnn.readNet(yolov3.config_path, yolov3.weights_path)
net

< cv2.dnn.Net 0x7f56780adeb0>